# The purpose of this notebook is to look at the data obtained in order to predict the rate of return

In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime
import yfinance as yf
from datetime import timedelta 

# Read Data
 * index contains an empty phrase, repeated in each line
 * Date - it is the date when the specified company received the return score (weekly)
 * Ticker_2 and Ticker - define the stock exchange symbol.
 * Name - the name of the company
 * Code - I do not know what it is
 * value - normalized rate of return
 
## Clear data 
* change data type
* rid off index

In [2]:
df=pd.read_csv('data_new.csv', sep=',', header=None, names=['Date', 'Ticker_2','Ticker','Category','Code', 'value'])
df = df.drop('Code', axis = 1)
df.head()

,Date,Ticker_2,Ticker,Category,value
10:01:54.481 77425 [77425-thread-2] INFO a.s.m.c.ConvictionImpl - CONVICTIONLISTTOPN,2004-02-11,SU,SU,Energy Minerals,0.953727
10:01:54.481 77425 [77425-thread-2] INFO a.s.m.c.ConvictionImpl - CONVICTIONLISTTOPN,2004-02-11,GGG,GGG,Producer Manufacturing,0.952753
10:01:54.481 77425 [77425-thread-2] INFO a.s.m.c.ConvictionImpl - CONVICTIONLISTTOPN,2004-02-11,WGR,WGR,Energy Minerals,0.947634
10:01:54.481 77425 [77425-thread-2] INFO a.s.m.c.ConvictionImpl - CONVICTIONLISTTOPN,2004-02-11,CWT,CWT,Utilities,0.934181
10:01:54.481 77425 [77425-thread-2] INFO a.s.m.c.ConvictionImpl - CONVICTIONLISTTOPN,2004-02-11,BLL,BLL,Process Industries,0.922862


In [3]:
df.shape

(37360, 5)

In [4]:
df.isnull().sum()

Date        0
Ticker_2    0
Ticker      0
Category    0
value       0
dtype: int64

In [5]:
df.dtypes

Date         object
Ticker_2     object
Ticker       object
Category     object
value       float64
dtype: object

In [6]:
df=df.convert_dtypes()
df.dtypes

Date         string
Ticker_2     string
Ticker       string
Category     string
value       Float64
dtype: object

In [7]:
df[(df.value == 0)]

,Date,Ticker_2,Ticker,Category,value


In [8]:
df = df.reset_index(drop=True)
df.head()

,Date,Ticker_2,Ticker,Category,value
0,2004-02-11,SU,SU,Energy Minerals,0.953727
1,2004-02-11,GGG,GGG,Producer Manufacturing,0.952753
2,2004-02-11,WGR,WGR,Energy Minerals,0.947634
3,2004-02-11,CWT,CWT,Utilities,0.934181
4,2004-02-11,BLL,BLL,Process Industries,0.922862


### I wanna check if one Tiker have more than one Code 
no there isn't any Tiket with two Codes

In [9]:
df.groupby(['Ticker','Category']).count()['Ticker_2'].reset_index().groupby('Ticker').count().value_counts()

Category  Ticker_2
1         1           1834
dtype: int64

In [10]:
print(f'Number of unique tickers/companies: {len(df.Ticker.unique())}')
print(f'Number of unique dates: {len(df.Date.unique())}')
print(f'So there should be: {len(df.Date.unique()) * len(df.Ticker.unique())} of rows')

Number of unique tickers/companies: 1834
Number of unique dates: 467
So there should be: 856478 of rows


### Description of the companies' data volumes
so mean rows for one company is 20, when all dates range is about 467

In [11]:
df.groupby('Ticker').count()['Date'].describe()

count    1834.000000
mean       20.370774
std        21.318179
min         1.000000
25%         5.000000
50%        13.000000
75%        28.000000
max       170.000000
Name: Date, dtype: float64

### Check Categories
As you can see, there is a Miscellaneous category that I will keep, despite there being no information on this company.

In [12]:
df.Category.value_counts()

Finance                   3778
Retail Trade              3146
Producer Manufacturing    3114
Utilities                 2972
Electronic Technology     2926
Consumer Non-Durables     2887
Consumer Services         2593
Process Industries        2509
Technology Services       2480
Health Technology         1403
Consumer Durables         1396
Industrial Services       1376
Energy Minerals           1296
Distribution Services     1250
Commercial Services       1176
Transportation            1174
Non-Energy Minerals        858
Health Services            530
Communications             431
Miscellaneous               65
Name: Category, dtype: Int64

# Data expansion 
As you can see from the data, it's hard to build a model from it, the variables are categorical in nature, plus they are unique 
that's why I decided to add information about the closing price of the company on a particular day, I will use yahoofinance API for this.

Due to a problem with missing data. Yahoofinance API has gaps of a few days, I fill them by searching for the closest existing data and assign it to the search.

In [13]:
def str_into_dt_or_timestamp(doc:str): 
    return datetime.timestamp(datetime.strptime(doc, '%Y-%m-%d'))

def clear_df_ML_finance(data_name): 
    df = pd.read_csv('data_new.csv', sep=',', header=None, names=['Date', 'Ticker_2', 'Ticker', 'Category', 'Code', 'Value'])
    df = df.dropna()
    df = df.convert_dtypes()
    df = df.drop('Ticker_2',axis=1)
    df = df.reset_index(drop=True)
    df['Timestamp'] = df.Date.apply(lambda x: str_into_dt_or_timestamp(x))
    df = df[ ~(df.Value == 0)]
    return df

In [14]:
df = clear_df_ML_finance('data_new')
df.head(10)

,Date,Ticker,Category,Code,Value,Timestamp
0,2004-02-11,SU,Energy Minerals,GN63J3-R,0.953727,1.076454e+09
1,2004-02-11,GGG,Producer Manufacturing,H5490W-R,0.952753,1.076454e+09
2,2004-02-11,WGR,Energy Minerals,V0622Q-R,0.947634,1.076454e+09
3,2004-02-11,CWT,Utilities,GSWXLY-R,0.934181,1.076454e+09
4,2004-02-11,BLL,Process Industries,VFT0VQ-R,0.922862,1.076454e+09
5,2004-02-11,APA,Energy Minerals,DMX4QY-R,0.912117,1.076454e+09
6,2004-02-11,JW.B,Consumer Services,D0MCMY-R,0.906333,1.076454e+09
7,2004-02-11,MATX,Transportation,QVP8YX-R,0.866946,1.076454e+09
8,2004-02-11,ROST,Retail Trade,FJ4NDH-R,0.864789,1.076454e+09
9,2004-02-11,AXL,Producer Manufacturing,W3DV77-R,0.861478,1.076454e+09


In [87]:
def get_data_around_date(data, date, days_around = 6):
    for i in range(days_around):
        output = data.loc[data.Date == datetime.strptime(date, '%Y-%m-%d') + timedelta(days = i)]
        if not output.empty:
            output['Date'] = date
            return output[['Code', 'Date', 'Close']]
        output = data.loc[data.Date == datetime.strptime(date, '%Y-%m-%d') - timedelta(days = i)]   
        if not output.empty:
            output['Date'] = date
            return output[['Code', 'Date', 'Close']]
    return pd.DataFrame()

def collect_data(stock_code, start, end, weeks):
    df = pd.DataFrame()
    for code in stock_code:
        collect = yf.download(code, 
                              start = start, 
                              end = end, 
                              progress = False,
                              interval = "1d",
        )
        collect = collect.reset_index()
        collect['Code'] = code
        collect_df = collect[(collect.Date.isin(weeks))][['Code', 'Date', 'Close']]
        for date in weeks:
            if date not in np.array(collect_df['Date'].astype(str)):
                collect_df = pd.concat([collect_df, get_data_around_date(collect, date)])
        df = pd.concat([df, collect_df])
    df['Date'] = df['Date'].astype(str).apply(lambda x: x.split()[0])
    df = df.drop_duplicates()
    return df
min_date = df.Date.min()
max_date = df.Date.max()
yf_data = collect_data(df.Short.unique(), min_date, max_date, df.Date.unique())
yf_data

,Code,Date,Close
0,SU,2004-02-11,13.285000
1,SU,2004-02-25,12.660000
2,SU,2004-03-10,13.465000
3,SU,2004-03-24,13.255000
4,SU,2004-04-07,13.130000
...,...,...,...
590986,AGCO,2021-12-29,115.889999
590987,AGCO,2022-01-12,123.750000
590988,AGCO,2022-01-26,116.370003
590989,AGCO,2018-12-05,58.189999


### Number of rows is smaller than 856478 but it's depend of download data form API

In [17]:
yf_data.groupby('Code').count()

,Date,Close
Code,,
A,467,467
AAIC,467,467
AAP,467,467
AAPL,467,467
AAT,287,287
...,...,...
YLWDF,236,236
YUM,467,467
ZBRA,467,467


# Marge data

In [18]:
df_new = df.merge(yf_data, left_on = ['Ticker', 'Date'], right_on = ['Code', 'Date'], how = 'inner')
df_new = df_new.drop('Code_y', axis = 1)
df_new = df_new.rename(columns = {'Code_x': 'Code'})
df_new

,Date,Ticker,Category,Code,Value,Timestamp,Close
0,2004-02-11,SU,Energy Minerals,GN63J3-R,0.953727,1.076454e+09,13.285000
1,2004-02-11,GGG,Producer Manufacturing,H5490W-R,0.952753,1.076454e+09,9.388889
2,2004-02-11,CWT,Utilities,GSWXLY-R,0.934181,1.076454e+09,14.720000
3,2004-02-11,BLL,Process Industries,VFT0VQ-R,0.922862,1.076454e+09,8.095000
4,2004-02-11,APA,Energy Minerals,DMX4QY-R,0.912117,1.076454e+09,39.830002
...,...,...,...,...,...,...,...
30556,2022-02-09,PEP,Consumer Non-Durables,PPCTFP-R,0.701507,1.644361e+09,172.020004
30557,2022-02-09,SSNC,Technology Services,G92RX2-R,0.701123,1.644361e+09,80.730003
30558,2022-02-09,GEF,Process Industries,MPX0N4-R,0.697954,1.644361e+09,57.880001
30559,2022-02-09,DPZ,Consumer Services,F05QG0-R,0.697741,1.644361e+09,438.730011


# Take care of outliers and save data

In [19]:
df_new = df_new.drop('Code', axis = 1)
df_new

,Date,Ticker,Category,Value,Timestamp,Close
0,2004-02-11,SU,Energy Minerals,0.953727,1.076454e+09,13.285000
1,2004-02-11,GGG,Producer Manufacturing,0.952753,1.076454e+09,9.388889
2,2004-02-11,CWT,Utilities,0.934181,1.076454e+09,14.720000
3,2004-02-11,BLL,Process Industries,0.922862,1.076454e+09,8.095000
4,2004-02-11,APA,Energy Minerals,0.912117,1.076454e+09,39.830002
...,...,...,...,...,...,...
30556,2022-02-09,PEP,Consumer Non-Durables,0.701507,1.644361e+09,172.020004
30557,2022-02-09,SSNC,Technology Services,0.701123,1.644361e+09,80.730003
30558,2022-02-09,GEF,Process Industries,0.697954,1.644361e+09,57.880001
30559,2022-02-09,DPZ,Consumer Services,0.697741,1.644361e+09,438.730011


In [22]:
df_new.Ticker.value_counts().describe()

count    1374.000000
mean       22.242358
std        22.606683
min         1.000000
25%         6.000000
50%        14.000000
75%        31.750000
max       170.000000
Name: Ticker, dtype: float64

In [23]:
Ticker_num = df_new['Ticker'].value_counts()
Ticker_num = df_new[df_new['Ticker'].isin(Ticker_num[Ticker_num >= 14].index)].copy()
Ticker_num

,Date,Ticker,Category,Value,Timestamp,Close
2,2004-02-11,CWT,Utilities,0.934181,1.076454e+09,14.720000
3,2004-02-11,BLL,Process Industries,0.922862,1.076454e+09,8.095000
4,2004-02-11,APA,Energy Minerals,0.912117,1.076454e+09,39.830002
5,2004-02-11,MATX,Transportation,0.866946,1.076454e+09,16.394106
6,2004-02-11,ROST,Retail Trade,0.864789,1.076454e+09,7.712500
...,...,...,...,...,...,...
30554,2022-02-09,SLGN,Process Industries,0.709506,1.644361e+09,43.750000
30556,2022-02-09,PEP,Consumer Non-Durables,0.701507,1.644361e+09,172.020004
30557,2022-02-09,SSNC,Technology Services,0.701123,1.644361e+09,80.730003
30558,2022-02-09,GEF,Process Industries,0.697954,1.644361e+09,57.880001


In [24]:
Ticker_num.to_csv('Prepared_data.csv')